In [ ]:
#Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole

from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error
from sklearn.model_selection import KFold, RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [ ]:
#Import dataset
df = pd.read_excel("training_data.xlsx")

#Generate molecules
df['ROMol'] = df.apply(lambda x: Chem.MolFromSmiles(x['SMILES']), axis=1)

#Attach RDKit descriptors
from rdkit.Chem.EState import EState
df['MaxEState'] = df.apply(lambda x: EState.MaxEStateIndex(x['ROMol']),axis=1)
df['MinEState'] = df.apply(lambda x: EState.MinEStateIndex(x['ROMol']),axis=1)
df['MaxAbsEState'] = df.apply(lambda x: EState.MaxAbsEStateIndex(x['ROMol']),axis=1)
df['MinAbsEState'] = df.apply(lambda x: EState.MinAbsEStateIndex(x['ROMol']),axis=1)

from rdkit.Chem import QED
df['qed'] = df.apply(lambda x: QED.qed(x['ROMol']),axis=1)

from rdkit.Chem import Descriptors
df['MolWt'] = df.apply(lambda x: Descriptors.MolWt(x['ROMol']),axis=1)
df['HeavyAtomMolWt'] = df.apply(lambda x: Descriptors.HeavyAtomMolWt(x['ROMol']),axis=1)
df['ExactMolWt'] = df.apply(lambda x: Descriptors.ExactMolWt(x['ROMol']),axis=1)
df['NumValenceElectrons'] = df.apply(lambda x: Descriptors.NumValenceElectrons(x['ROMol']),axis=1)
df['NumRadicalElectrons'] = df.apply(lambda x: Descriptors.NumRadicalElectrons(x['ROMol']),axis=1)
df['MaxPartialCharge'] = df.apply(lambda x: Descriptors.MaxPartialCharge(x['ROMol']),axis=1)
df['MinPartialCharge'] = df.apply(lambda x: Descriptors.MinPartialCharge(x['ROMol']),axis=1)
df['MaxAbsPartialCharge'] = df.apply(lambda x: Descriptors.MaxAbsPartialCharge(x['ROMol']),axis=1)
df['MinAbsPartialCharge'] = df.apply(lambda x: Descriptors.MinAbsPartialCharge(x['ROMol']),axis=1)
df['FpDensityMorgan1'] = df.apply(lambda x: Descriptors.FpDensityMorgan1(x['ROMol']),axis=1)
df['FpDensityMorgan2'] = df.apply(lambda x: Descriptors.FpDensityMorgan2(x['ROMol']),axis=1)
df['FpDensityMorgan3'] = df.apply(lambda x: Descriptors.FpDensityMorgan3(x['ROMol']),axis=1)

from rdkit.Chem import GraphDescriptors
df['BalabanJ'] = df.apply(lambda x: GraphDescriptors.BalabanJ(x['ROMol']),axis=1)
df['BertzCT'] = df.apply(lambda x: GraphDescriptors.BertzCT(x['ROMol']),axis=1)
df['Chi0'] = df.apply(lambda x: GraphDescriptors.Chi0(x['ROMol']),axis=1)
df['Chi0n'] = df.apply(lambda x: GraphDescriptors.Chi0n(x['ROMol']),axis=1)
df['Chi0v'] = df.apply(lambda x: GraphDescriptors.Chi0v(x['ROMol']),axis=1)
df['Chi1'] = df.apply(lambda x: GraphDescriptors.Chi1(x['ROMol']),axis=1)
df['Chi1n'] = df.apply(lambda x: GraphDescriptors.Chi1n(x['ROMol']),axis=1)
df['Chi1v'] = df.apply(lambda x: GraphDescriptors.Chi1v(x['ROMol']),axis=1)
df['Chi2n'] = df.apply(lambda x: GraphDescriptors.Chi2n(x['ROMol']),axis=1)
df['Chi2v'] = df.apply(lambda x: GraphDescriptors.Chi2v(x['ROMol']),axis=1)
df['Chi3n'] = df.apply(lambda x: GraphDescriptors.Chi3n(x['ROMol']),axis=1)
df['Chi3v'] = df.apply(lambda x: GraphDescriptors.Chi3v(x['ROMol']),axis=1)
df['Chi4n'] = df.apply(lambda x: GraphDescriptors.Chi4n(x['ROMol']),axis=1)
df['Chi4v'] = df.apply(lambda x: GraphDescriptors.Chi4v(x['ROMol']),axis=1)
df['HallKierAlpha'] = df.apply(lambda x: GraphDescriptors.HallKierAlpha(x['ROMol']),axis=1)
df['Ipc'] = df.apply(lambda x: GraphDescriptors.Ipc(x['ROMol']),axis=1)
df['Kappa1'] = df.apply(lambda x: GraphDescriptors.Kappa1(x['ROMol']),axis=1)
df['Kappa2'] = df.apply(lambda x: GraphDescriptors.Kappa2(x['ROMol']),axis=1)
df['Kappa3'] = df.apply(lambda x: GraphDescriptors.Kappa3(x['ROMol']),axis=1)

from rdkit.Chem import rdMolDescriptors
df['LabuteASA'] = df.apply(lambda x: rdMolDescriptors.CalcLabuteASA(x['ROMol']),axis=1)
df['TPSA'] = df.apply(lambda x: rdMolDescriptors.CalcTPSA(x['ROMol']),axis=1)

from rdkit.Chem import MolSurf
df['PEOE_VSA1'] = df.apply(lambda x: MolSurf.PEOE_VSA1(x['ROMol']),axis=1)
df['PEOE_VSA10'] = df.apply(lambda x: MolSurf.PEOE_VSA10(x['ROMol']),axis=1)
df['PEOE_VSA11'] = df.apply(lambda x: MolSurf.PEOE_VSA11(x['ROMol']),axis=1)
df['PEOE_VSA12'] = df.apply(lambda x: MolSurf.PEOE_VSA12(x['ROMol']),axis=1)
df['PEOE_VSA13'] = df.apply(lambda x: MolSurf.PEOE_VSA11(x['ROMol']),axis=1)
df['PEOE_VSA14'] = df.apply(lambda x: MolSurf.PEOE_VSA12(x['ROMol']),axis=1)
df['PEOE_VSA2'] = df.apply(lambda x: MolSurf.PEOE_VSA2(x['ROMol']),axis=1)
df['PEOE_VSA3'] = df.apply(lambda x: MolSurf.PEOE_VSA3(x['ROMol']),axis=1)
df['PEOE_VSA4'] = df.apply(lambda x: MolSurf.PEOE_VSA4(x['ROMol']),axis=1)
df['PEOE_VSA5'] = df.apply(lambda x: MolSurf.PEOE_VSA5(x['ROMol']),axis=1)
df['PEOE_VSA6'] = df.apply(lambda x: MolSurf.PEOE_VSA6(x['ROMol']),axis=1)
df['PEOE_VSA7'] = df.apply(lambda x: MolSurf.PEOE_VSA7(x['ROMol']),axis=1)
df['PEOE_VSA8'] = df.apply(lambda x: MolSurf.PEOE_VSA8(x['ROMol']),axis=1)
df['PEOE_VSA9'] = df.apply(lambda x: MolSurf.PEOE_VSA9(x['ROMol']),axis=1)
df['SMR_VSA1'] = df.apply(lambda x: MolSurf.SMR_VSA1(x['ROMol']),axis=1)
df['SMR_VSA10'] = df.apply(lambda x: MolSurf.SMR_VSA10(x['ROMol']),axis=1)
df['SMR_VSA2'] = df.apply(lambda x: MolSurf.SMR_VSA2(x['ROMol']),axis=1)
df['SMR_VSA3'] = df.apply(lambda x: MolSurf.SMR_VSA3(x['ROMol']),axis=1)
df['SMR_VSA4'] = df.apply(lambda x: MolSurf.SMR_VSA4(x['ROMol']),axis=1)
df['SMR_VSA5'] = df.apply(lambda x: MolSurf.SMR_VSA5(x['ROMol']),axis=1)
df['SMR_VSA6'] = df.apply(lambda x: MolSurf.SMR_VSA6(x['ROMol']),axis=1)
df['SMR_VSA7'] = df.apply(lambda x: MolSurf.SMR_VSA7(x['ROMol']),axis=1)
df['SMR_VSA8'] = df.apply(lambda x: MolSurf.SMR_VSA8(x['ROMol']),axis=1)
df['SMR_VSA9'] = df.apply(lambda x: MolSurf.SMR_VSA9(x['ROMol']),axis=1)
df['SlogP_VSA1'] = df.apply(lambda x: MolSurf.SlogP_VSA1(x['ROMol']),axis=1)
df['SlogP_VSA10'] = df.apply(lambda x: MolSurf.SlogP_VSA10(x['ROMol']),axis=1)
df['SlogP_VSA2'] = df.apply(lambda x: MolSurf.SlogP_VSA2(x['ROMol']),axis=1)
df['SlogP_VSA3'] = df.apply(lambda x: MolSurf.SlogP_VSA3(x['ROMol']),axis=1)
df['SlogP_VSA4'] = df.apply(lambda x: MolSurf.SlogP_VSA4(x['ROMol']),axis=1)
df['SlogP_VSA5'] = df.apply(lambda x: MolSurf.SlogP_VSA5(x['ROMol']),axis=1)
df['SlogP_VSA6'] = df.apply(lambda x: MolSurf.SlogP_VSA6(x['ROMol']),axis=1)
df['SlogP_VSA7'] = df.apply(lambda x: MolSurf.SlogP_VSA7(x['ROMol']),axis=1)
df['SlogP_VSA8'] = df.apply(lambda x: MolSurf.SlogP_VSA8(x['ROMol']),axis=1)
df['SlogP_VSA9'] = df.apply(lambda x: MolSurf.SlogP_VSA9(x['ROMol']),axis=1)
df['SlogP_VSA11'] = df.apply(lambda x: MolSurf.SlogP_VSA11(x['ROMol']),axis=1)
df['SlogP_VSA12'] = df.apply(lambda x: MolSurf.SlogP_VSA12(x['ROMol']),axis=1)

from rdkit.Chem.EState import EState_VSA
df['EState_VSA1'] = df.apply(lambda x: EState_VSA.EState_VSA1(x['ROMol']),axis=1)
df['EState_VSA10'] = df.apply(lambda x: EState_VSA.EState_VSA10(x['ROMol']),axis=1)
df['EState_VSA11'] = df.apply(lambda x: EState_VSA.EState_VSA11(x['ROMol']),axis=1)
df['EState_VSA2'] = df.apply(lambda x: EState_VSA.EState_VSA2(x['ROMol']),axis=1)
df['EState_VSA3'] = df.apply(lambda x: EState_VSA.EState_VSA3(x['ROMol']),axis=1)
df['EState_VSA4'] = df.apply(lambda x: EState_VSA.EState_VSA4(x['ROMol']),axis=1)
df['EState_VSA5'] = df.apply(lambda x: EState_VSA.EState_VSA5(x['ROMol']),axis=1)
df['EState_VSA6'] = df.apply(lambda x: EState_VSA.EState_VSA6(x['ROMol']),axis=1)
df['EState_VSA7'] = df.apply(lambda x: EState_VSA.EState_VSA7(x['ROMol']),axis=1)
df['EState_VSA8'] = df.apply(lambda x: EState_VSA.EState_VSA8(x['ROMol']),axis=1)
df['EState_VSA9'] = df.apply(lambda x: EState_VSA.EState_VSA9(x['ROMol']),axis=1)
df['VSA_Estate1'] = df.apply(lambda x: EState_VSA.VSA_EState1(x['ROMol']),axis=1)
df['VSA_Estate10'] = df.apply(lambda x: EState_VSA.VSA_EState10(x['ROMol']),axis=1)
df['VSA_Estate2'] = df.apply(lambda x: EState_VSA.VSA_EState2(x['ROMol']),axis=1)
df['VSA_Estate3'] = df.apply(lambda x: EState_VSA.VSA_EState3(x['ROMol']),axis=1)
df['VSA_Estate4'] = df.apply(lambda x: EState_VSA.VSA_EState4(x['ROMol']),axis=1)
df['VSA_Estate5'] = df.apply(lambda x: EState_VSA.VSA_EState5(x['ROMol']),axis=1)
df['VSA_Estate6'] = df.apply(lambda x: EState_VSA.VSA_EState6(x['ROMol']),axis=1)
df['VSA_Estate7'] = df.apply(lambda x: EState_VSA.VSA_EState7(x['ROMol']),axis=1)
df['VSA_Estate8'] = df.apply(lambda x: EState_VSA.VSA_EState8(x['ROMol']),axis=1)
df['VSA_Estate9'] = df.apply(lambda x: EState_VSA.VSA_EState9(x['ROMol']),axis=1)

df = df.copy()

from rdkit.Chem import Lipinski
df['FractionCSP3'] = df.apply(lambda x: Lipinski.FractionCSP3(x['ROMol']),axis=1)
df['HeavyAtomCount'] = df.apply(lambda x: Lipinski.HeavyAtomCount(x['ROMol']),axis=1)
df['NHOHCount'] = df.apply(lambda x: Lipinski.NHOHCount(x['ROMol']),axis=1)
df['NOCount'] = df.apply(lambda x: Lipinski.NOCount(x['ROMol']),axis=1)
df['NumAliphaticCarbocycles'] = df.apply(lambda x: Lipinski.NumAliphaticCarbocycles(x['ROMol']),axis=1)
df['NumAliphaticHeterocycles'] = df.apply(lambda x: Lipinski.NumAliphaticHeterocycles(x['ROMol']),axis=1)
df['NumAliphaticRings'] = df.apply(lambda x: Lipinski.NumAliphaticRings(x['ROMol']),axis=1)
df['NumAromaticCarbocycles'] = df.apply(lambda x: Lipinski.NumAromaticCarbocycles(x['ROMol']),axis=1)
df['NumAromaticHeterocycles'] = df.apply(lambda x: Lipinski.NumAromaticHeterocycles(x['ROMol']),axis=1)
df['NumAromaticRings'] = df.apply(lambda x: Lipinski.NumAromaticRings(x['ROMol']),axis=1)
df['NumHAcceptors'] = df.apply(lambda x: Lipinski.NumHAcceptors(x['ROMol']),axis=1)
df['NumHDonors'] = df.apply(lambda x: Lipinski.NumHDonors(x['ROMol']),axis=1)
df['NumHeteroatoms'] = df.apply(lambda x: Lipinski.NumHeteroatoms(x['ROMol']),axis=1)
df['NumRotatableBonds'] = df.apply(lambda x: Lipinski.NumRotatableBonds(x['ROMol']),axis=1)
df['NumSaturatedCarbocycles'] = df.apply(lambda x: Lipinski.NumSaturatedCarbocycles(x['ROMol']),axis=1)
df['NumSaturatedHeterocycles'] = df.apply(lambda x: Lipinski.NumSaturatedHeterocycles(x['ROMol']),axis=1)
df['NumSaturatedRings'] = df.apply(lambda x: Lipinski.NumSaturatedRings(x['ROMol']),axis=1)
df['RingCount'] = df.apply(lambda x: Lipinski.RingCount(x['ROMol']),axis=1)

from rdkit.Chem import Crippen
df['MolLogP'] = df.apply(lambda x: Crippen.MolLogP(x['ROMol']),axis=1)
df['MolMR'] = df.apply(lambda x: Crippen.MolMR(x['ROMol']),axis=1)

from rdkit.Chem import Fragments
df['fr_Al_COO'] = df.apply(lambda x: Fragments.fr_Al_COO(x['ROMol']),axis=1)
df['fr_Al_OH'] = df.apply(lambda x: Fragments.fr_Al_OH(x['ROMol']),axis=1)
df['fr_Al_OH_noTert'] = df.apply(lambda x: Fragments.fr_Al_OH_noTert(x['ROMol']),axis=1)
df['fr_ArN'] = df.apply(lambda x: Fragments.fr_ArN(x['ROMol']),axis=1)
df['fr_Ar_COO'] = df.apply(lambda x: Fragments.fr_Ar_COO(x['ROMol']),axis=1)
df['fr_Ar_N'] = df.apply(lambda x: Fragments.fr_Ar_N(x['ROMol']),axis=1)
df['fr_Ar_NH'] = df.apply(lambda x: Fragments.fr_Ar_NH(x['ROMol']),axis=1)
df['fr_Ar_OH'] = df.apply(lambda x: Fragments.fr_Ar_OH(x['ROMol']),axis=1)
df['fr_COO'] = df.apply(lambda x: Fragments.fr_COO(x['ROMol']),axis=1)
df['fr_COO2'] = df.apply(lambda x: Fragments.fr_COO2(x['ROMol']),axis=1)
df['fr_C_O'] = df.apply(lambda x: Fragments.fr_C_O(x['ROMol']),axis=1)
df['fr_C_O_noCOO'] = df.apply(lambda x: Fragments.fr_C_O_noCOO(x['ROMol']),axis=1)
df['fr_C_S'] = df.apply(lambda x: Fragments.fr_C_S(x['ROMol']),axis=1)
df['fr_HOCCN'] = df.apply(lambda x: Fragments.fr_HOCCN(x['ROMol']),axis=1)
df['fr_Imine'] = df.apply(lambda x: Fragments.fr_Imine(x['ROMol']),axis=1)
df['fr_NH0'] = df.apply(lambda x: Fragments.fr_NH0(x['ROMol']),axis=1)
df['fr_NH1'] = df.apply(lambda x: Fragments.fr_NH1(x['ROMol']),axis=1)
df['fr_NH2'] = df.apply(lambda x: Fragments.fr_NH2(x['ROMol']),axis=1)
df['fr_N_O'] = df.apply(lambda x: Fragments.fr_N_O(x['ROMol']),axis=1)
df['fr_Ndealkylation1'] = df.apply(lambda x: Fragments.fr_Ndealkylation1(x['ROMol']),axis=1)
df['fr_Ndealkylation2'] = df.apply(lambda x: Fragments.fr_Ndealkylation2(x['ROMol']),axis=1)
df['fr_Nhpyrrole'] = df.apply(lambda x: Fragments.fr_Nhpyrrole(x['ROMol']),axis=1)
df['fr_SH'] = df.apply(lambda x: Fragments.fr_SH(x['ROMol']),axis=1)
df['fr_aldehyde'] = df.apply(lambda x: Fragments.fr_aldehyde(x['ROMol']),axis=1)
df['fr_alkyl_carbamate'] = df.apply(lambda x: Fragments.fr_alkyl_carbamate(x['ROMol']),axis=1)
df['fr_alkyl_halide'] = df.apply(lambda x: Fragments.fr_alkyl_halide(x['ROMol']),axis=1)
df['fr_allylic_oxid'] = df.apply(lambda x: Fragments.fr_allylic_oxid(x['ROMol']),axis=1)
df['fr_amide'] = df.apply(lambda x: Fragments.fr_amide(x['ROMol']),axis=1)
df['fr_amidine'] = df.apply(lambda x: Fragments.fr_amidine(x['ROMol']),axis=1)
df['fr_aniline'] = df.apply(lambda x: Fragments.fr_aniline(x['ROMol']),axis=1)
df['fr_aryl_methyl'] = df.apply(lambda x: Fragments.fr_aryl_methyl(x['ROMol']),axis=1)
df['fr_azide'] = df.apply(lambda x: Fragments.fr_azide(x['ROMol']),axis=1)
df['fr_azo'] = df.apply(lambda x: Fragments.fr_azo(x['ROMol']),axis=1)
df['fr_barbitur'] = df.apply(lambda x: Fragments.fr_barbitur(x['ROMol']),axis=1)
df['fr_benzene'] = df.apply(lambda x: Fragments.fr_benzene(x['ROMol']),axis=1)
df = df.copy()
df['fr_benzodiazepine'] = df.apply(lambda x: Fragments.fr_benzodiazepine(x['ROMol']),axis=1)
df['fr_bicyclic'] = df.apply(lambda x: Fragments.fr_bicyclic(x['ROMol']),axis=1)
df['fr_diazo'] = df.apply(lambda x: Fragments.fr_diazo(x['ROMol']),axis=1)
df['fr_dihydropyridine'] = df.apply(lambda x: Fragments.fr_dihydropyridine(x['ROMol']),axis=1)
df['fr_epoxide'] = df.apply(lambda x: Fragments.fr_epoxide(x['ROMol']),axis=1)
df['fr_ester'] = df.apply(lambda x: Fragments.fr_ester(x['ROMol']),axis=1)
df['fr_ether'] = df.apply(lambda x: Fragments.fr_ether(x['ROMol']),axis=1)
df['fr_furan'] = df.apply(lambda x: Fragments.fr_furan(x['ROMol']),axis=1)
df['fr_guanido'] = df.apply(lambda x: Fragments.fr_guanido(x['ROMol']),axis=1)
df['fr_halogen'] = df.apply(lambda x: Fragments.fr_halogen(x['ROMol']),axis=1)
df['fr_hdrzine'] = df.apply(lambda x: Fragments.fr_hdrzine(x['ROMol']),axis=1)
df['fr_hdrzone'] = df.apply(lambda x: Fragments.fr_hdrzone(x['ROMol']),axis=1)
df['fr_imidazole'] = df.apply(lambda x: Fragments.fr_imidazole(x['ROMol']),axis=1)
df['fr_imide'] = df.apply(lambda x: Fragments.fr_imide(x['ROMol']),axis=1)
df['fr_isocyan'] = df.apply(lambda x: Fragments.fr_isocyan(x['ROMol']),axis=1)
df['fr_isothiocyan'] = df.apply(lambda x: Fragments.fr_isothiocyan(x['ROMol']),axis=1)
df['fr_ketone'] = df.apply(lambda x: Fragments.fr_ketone(x['ROMol']),axis=1)
df['fr_ketone_Topliss'] = df.apply(lambda x: Fragments.fr_ketone_Topliss(x['ROMol']),axis=1)
df['fr_lactam'] = df.apply(lambda x: Fragments.fr_lactam(x['ROMol']),axis=1)
df['fr_lactone'] = df.apply(lambda x: Fragments.fr_lactone(x['ROMol']),axis=1)
df['fr_methoxy'] = df.apply(lambda x: Fragments.fr_methoxy(x['ROMol']),axis=1)
df['fr_morpholine'] = df.apply(lambda x: Fragments.fr_morpholine(x['ROMol']),axis=1)
df['fr_nitrile'] = df.apply(lambda x: Fragments.fr_nitrile(x['ROMol']),axis=1)
df['fr_nitro'] = df.apply(lambda x: Fragments.fr_nitro(x['ROMol']),axis=1)
df['fr_nitro_arom'] = df.apply(lambda x: Fragments.fr_nitro_arom(x['ROMol']),axis=1)
df['fr_nitro_arom_nonortho'] = df.apply(lambda x: Fragments.fr_nitro_arom_nonortho(x['ROMol']),axis=1)
df['fr_nitroso'] = df.apply(lambda x: Fragments.fr_nitroso(x['ROMol']),axis=1)
df['fr_oxazole'] = df.apply(lambda x: Fragments.fr_oxazole(x['ROMol']),axis=1)
df['fr_oxime'] = df.apply(lambda x: Fragments.fr_oxime(x['ROMol']),axis=1)
df['fr_para_hydroxylation'] = df.apply(lambda x: Fragments.fr_para_hydroxylation(x['ROMol']),axis=1)
df['fr_phenol'] = df.apply(lambda x: Fragments.fr_phenol(x['ROMol']),axis=1)
df['fr_phenol_noOrthoHbond'] = df.apply(lambda x: Fragments.fr_phenol_noOrthoHbond(x['ROMol']),axis=1)
df['fr_phos_acid'] = df.apply(lambda x: Fragments.fr_phos_acid(x['ROMol']),axis=1)
df['fr_phos_ester'] = df.apply(lambda x: Fragments.fr_phos_ester(x['ROMol']),axis=1)
df['fr_piperdine'] = df.apply(lambda x: Fragments.fr_piperdine(x['ROMol']),axis=1)
df['fr_piperzine'] = df.apply(lambda x: Fragments.fr_piperzine(x['ROMol']),axis=1)
df['fr_priamide'] = df.apply(lambda x: Fragments.fr_priamide(x['ROMol']),axis=1)
df['fr_prisulfonamd'] = df.apply(lambda x: Fragments.fr_prisulfonamd(x['ROMol']),axis=1)
df['fr_pyridine'] = df.apply(lambda x: Fragments.fr_pyridine(x['ROMol']),axis=1)
df['fr_quatN'] = df.apply(lambda x: Fragments.fr_quatN(x['ROMol']),axis=1)
df['fr_sulfide'] = df.apply(lambda x: Fragments.fr_sulfide(x['ROMol']),axis=1)
df['fr_sulfonamd'] = df.apply(lambda x: Fragments.fr_sulfonamd(x['ROMol']),axis=1)
df['fr_sulfone'] = df.apply(lambda x: Fragments.fr_sulfone(x['ROMol']),axis=1)
df['fr_term_acetylene'] = df.apply(lambda x: Fragments.fr_term_acetylene(x['ROMol']),axis=1)
df['fr_tetrazole'] = df.apply(lambda x: Fragments.fr_tetrazole(x['ROMol']),axis=1)
df['fr_thiazole'] = df.apply(lambda x: Fragments.fr_thiazole(x['ROMol']),axis=1)
df['fr_thiocyan'] = df.apply(lambda x: Fragments.fr_thiocyan(x['ROMol']),axis=1)
df['fr_thiophene'] = df.apply(lambda x: Fragments.fr_thiophene(x['ROMol']),axis=1)
df['fr_unbrch_alkane'] = df.apply(lambda x: Fragments.fr_unbrch_alkane(x['ROMol']),axis=1)
df['fr_urea'] = df.apply(lambda x: Fragments.fr_urea(x['ROMol']),axis=1)

#Split dataset
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,5:],df['value'],test_size=0.2,random_state=42)
X_train = np.array(X_train).astype(float)
X_train = np.nan_to_num(X_train,nan=0.0,posinf=3.4e+38,neginf=-3.4e+38)
X_test = np.array(X_test).astype(float)
X_test = np.nan_to_num(X_test,nan=0.0,posinf=3.4e+38,neginf=-3.4e+38)
y_train = np.array(y_train).astype(float)
y_test = np.array(y_test).astype(float)

#Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Random Forest (RF)

In [ ]:
#Hyperparameter tuning of RF model

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
#Number of features to consider at every split
max_features = ['log2', 'sqrt',1.0]
#Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
#Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
#Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
#Method of selecting samples for training each tree
bootstrap = [True, False] 
#Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

#Initialise model
rf = RandomForestRegressor()
#Implement random search of parameters with 5-fold validation
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=1, random_state=0)
# Fit the random search model
rf_random.fit(X_train, y_train)

print(rf_random.best_params_)

In [ ]:
#Implement grid search hyperparameter tuning

param_grid = {'n_estimators' : [1000,1100,1200],
'max_features' : ['sqrt'],
'max_depth' : [70,80,90],
'min_samples_split' : [2,3],
'min_samples_leaf' : [1],
'bootstrap' : [False]
}

#Initialise model
rf = RandomForestRegressor()
#Implment grid search
rf_grid = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, verbose = 1)
rf_grid.fit(X_train,y_train)
print(rf_grid.best_params_)

In [ ]:
#perform 5-fold cross validation with parameters
kf = KFold(n_splits=5)
r2, mae, rmse = [], [], []
for train,test in kf.split(X_train,y_train):
    regr = RandomForestRegressor(n_estimators = 1000, max_depth = 90, min_samples_split = 2, max_features='sqrt', min_samples_leaf = 1, bootstrap = False, random_state = 0)
    regr.fit(X_train[train],y_train[train])
    y_pred = regr.predict(X_train[test])
    y_true = np.array(y_train[test])
    r2.append(r2_score(y_true,y_pred))
    mae.append(mean_absolute_error(y_true,y_pred))
    rmse.append(mean_squared_error(y_true,y_pred,squared=False))

    #plot data
    # y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
    # plt.scatter(y_true,y_pred)
    # plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black')
    # plt.annotate("r2 = {:.3f}".format(r2_score(y_true, y_pred)), (200, -50))
    # plt.xlabel("Actual Tg ($^\circ$C)")
    # plt.ylabel("Predicted Tg ($^\circ$C)")
    # plt.show()
    # del regr

print(r2)
print(mae)
print(rmse)
print(f"Average r2 score: {np.mean(r2)}\nAverage MAE: {np.mean(mae)}\nAverage RMSE: {np.mean(rmse)}")

In [ ]:
#test against unseen data
regr = RandomForestRegressor(n_estimators = 1000, max_depth = 70, min_samples_split = 2, max_features='sqrt', min_samples_leaf = 1, bootstrap = True, random_state = 123)
regr.fit(X_train,y_train)
y_pred = regr.predict(X_test)
y_true = np.array(y_test)

print(r2_score(y_true,y_pred))
print(mean_absolute_error(y_true,y_pred))
print(mean_squared_error(y_true,y_pred,squared=False))

In [ ]:
#plot data
y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
plt.scatter(y_true,y_pred,s=10,alpha=0.9)
plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black',alpha=0.7)
plt.xlabel("Actual Bandgap (eV)")
plt.ylabel("Predicted Bandgap (eV)")
plt.show()

Support Vector Machine (SVM)

In [ ]:
#hyperparameter tuning of SVM model

param_grid = {'kernel' : ['linear'],
              'C' : [0.1, 1],
              'gamma' : [0.001, 0.01, 0.01,'scale','auto'],
              'epsilon' : [0.001, 0.01, 0.1, 1]
              }

#Initialise model
svr = svm.SVR()
#Perform grid search
svr_grid = GridSearchCV(estimator = svr, param_grid = param_grid, cv = 3, verbose = 1)
#Fit grid search model
svr_grid.fit(X_train,y_train)

print(svr_grid.best_params_)

In [ ]:
#further tuning of SVM model

param_grid = {'kernel' : ['linear','rbf','poly','sigmoid'],
              'C' : [1,10],
              'gamma' : [0.001],
              'epsilon' : [0.001,0.01, 0.1]
              }

#Initialise model
svr = svm.SVR()
#Perform grid search
svr_grid = GridSearchCV(estimator = svr, param_grid = param_grid, cv = 5, verbose = 1)
#Fit grid search model
svr_grid.fit(X_train,y_train)

print(svr_grid.best_params_)

In [ ]:
#Perform 5-fold cross validation with parameters
kf = KFold(n_splits=5)
r2, mae, rmse = [], [], []
for train,test in kf.split(X_train,y_train):
    regr = svm.SVR(kernel="linear", C=0.1, tol=0.001, gamma=0.001,epsilon=0.01)
    regr.fit(X_train[train],y_train[train])
    y_pred = regr.predict(X_train[test])
    y_true = np.array(y_train[test])
    r2.append(r2_score(y_true,y_pred))
    mae.append(mean_absolute_error(y_true,y_pred))
    rmse.append(mean_squared_error(y_true,y_pred,squared=False))

    #plot data
    # y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
    # plt.scatter(y_true,y_pred)
    # plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black')
    # plt.annotate("r2 = {:.3f}".format(r2_score(y_true, y_pred)), (200, -50))
    # plt.xlabel("Actual Tg ($^\circ$C)")
    # plt.ylabel("Predicted Tg ($^\circ$C)")
    # plt.show()
   

print(r2)
print(mae)
print(rmse)
print(f"Average r2 score: {np.mean(r2)}\nAverage MAE: {np.mean(mae)}\nAverage RMSE: {np.mean(rmse)}")

In [ ]:
#Test model on unseen data
regr.fit(X_train,y_train)
y_pred = regr.predict(X_test)
y_true = np.array(y_test)

print(r2_score(y_true,y_pred))
print(mean_absolute_error(y_true,y_pred))
print(mean_squared_error(y_true,y_pred,squared=False))

In [ ]:
#plot data
y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
plt.scatter(y_true,y_pred,s=10,alpha=0.9)
plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black',alpha=0.7)
plt.xlabel("Actual Bandgap (eV)")
plt.ylabel("Predicted Bandgap (eV)")
plt.show()

Gaussian Process Regression (GPR)

In [ ]:
#import GPR model
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF

#initialise kernel
kernel = WhiteKernel(noise_level=0.1) + RBF(length_scale=10)

#perform 5-fold cross validation for MF
kf = KFold(n_splits=5)
r2, mae, rmse = [], [], []
for train,test in kf.split(X_train,y_train):
    regr = GaussianProcessRegressor(kernel=kernel,random_state=0)
    regr.fit(X_train[train],y_train[train])
    y_pred = regr.predict(X_train[test])
    y_true = np.array(y_train[test])
    r2.append(r2_score(y_true,y_pred))
    mae.append(mean_absolute_error(y_true,y_pred))
    rmse.append(mean_squared_error(y_true,y_pred,squared=False))

    #plot data
    # y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
    # plt.scatter(y_true,y_pred)
    # plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black')
    # plt.annotate("r2 = {:.3f}".format(r2_score(y_true, y_pred)), (200, -50))
    # plt.xlabel("Actual Tg ($^\circ$C)")
    # plt.ylabel("Predicted Tg ($^\circ$C)")
    # plt.show()
    # del regr

print(r2)
print(mae)
print(rmse)
print(f"Average r2 score: {np.mean(r2)}\nAverage MAE: {np.mean(mae)}\nAverage RMSE: {np.mean(rmse)}")

In [ ]:
#test against unseen data
regr.fit(X_train,y_train)
y_pred = regr.predict(X_test)
y_true = np.array(y_test)

print(r2_score(y_true,y_pred))
print(mean_absolute_error(y_true,y_pred))
print(mean_squared_error(y_true,y_pred,squared=False))

In [ ]:
#plot data
y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
plt.scatter(y_true,y_pred,s=10,alpha=0.9)
plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black',alpha=0.7)
# plt.annotate("r2 = {:.4f}".format(r2_score(y_true, y_pred)), (7.5, 1))
plt.xlabel("Actual Bandgap (eV)")
plt.ylabel("Predicted Bandgap (eV)")
plt.show()

Convolutional Neural Network (CNN)

In [ ]:
#import libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D

#define parameters
batch_size = 32
epochs = 100
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

#implement model for MF
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(200, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))
#compile model
model.compile(optimizer='adam', loss='mean_squared_error')

#run k-cross validation
kf = KFold(n_splits=5)
r2, mae, rmse = [], [], []
train_loss, val_loss = [], []
for train,test in kf.split(X_train,y_train):
    hist = model.fit(X_train[train],y_train[train],batch_size,epochs,verbose=0,validation_data=(X_train[test], y_train[test]))
    y_pred = model.predict(X_train[test])
    y_true = np.array(y_train[test])
    r2.append(r2_score(y_true,y_pred))
    mae.append(mean_absolute_error(y_true,y_pred))
    rmse.append(mean_squared_error(y_true,y_pred,squared=False))

    #record loss values
    train_loss.append(hist.history['loss'])
    val_loss.append(hist.history['val_loss'])

#learning curve
plt.figure(figsize=(10, 5))
plt.plot(np.mean(train_loss, axis=0), label='Training Loss')
plt.plot(np.mean(val_loss, axis=0), label='Validation Loss')
plt.title('Learning Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

print(f"Average r2 score: {np.mean(r2)}\nAverage MAE: {np.mean(mae)}\nAverage RMSE: {np.mean(rmse)}")

In [ ]:
#test against unseen data
model.fit(X_train,y_train,batch_size,epochs,verbose=0)
y_pred = model.predict(X_test)
y_true = np.array(y_test)

print(r2_score(y_true,y_pred))
print(mean_absolute_error(y_true,y_pred))
print(mean_squared_error(y_true,y_pred,squared=False))

In [ ]:
#plot data
y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
plt.scatter(y_true,y_pred,s=10,alpha=0.9)
plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black',alpha=0.7)
plt.xlabel("Actual Bandgap (eV)")
plt.ylabel("Predicted Bandgap (eV)")
plt.show()

Recurrent Neural Network (RNN)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

#define parameters
batch_size = 32
epochs = 100
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1]) 

#implment model for MF
model = Sequential()
model.add(LSTM(64, input_shape=(1, 200)))
model.add(Dense(1, activation='linear'))
#compile model
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])

#run k-cross validation
kf = KFold(n_splits=5)
r2, mae, rmse = [], [], []
train_loss, val_loss = [], []
for train,test in kf.split(X_train,y_train):
    hist = model.fit(X_train[train],y_train[train],batch_size,epochs,verbose=0,validation_data=(X_train[test], y_train[test]))
    y_pred = model.predict(X_train[test])
    y_true = np.array(y_train[test])
    r2.append(r2_score(y_true,y_pred))
    mae.append(mean_absolute_error(y_true,y_pred))
    rmse.append(mean_squared_error(y_true,y_pred,squared=False))

    #record loss values
    train_loss.append(hist.history['loss'])
    val_loss.append(hist.history['val_loss'])

#learning curve
plt.figure(figsize=(10, 5))
plt.plot(np.mean(train_loss, axis=0), label='Training Loss')
plt.plot(np.mean(val_loss, axis=0), label='Validation Loss')
plt.title('Learning Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

print(f"Average r2 score: {np.mean(r2)}\nAverage MAE: {np.mean(mae)}\nAverage RMSE: {np.mean(rmse)}")


In [ ]:
#test against unseen data
model.fit(X_train,y_train,batch_size,epochs,verbose=0)
y_pred = model.predict(X_test)
y_true = np.array(y_test)

print(r2_score(y_true,y_pred))
print(mean_absolute_error(y_true,y_pred))
print(mean_squared_error(y_true,y_pred,squared=False))

In [ ]:
#plot data
y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
plt.scatter(y_true,y_pred,s=10,alpha=0.9)
plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black',alpha=0.7)
plt.xlabel("Actual Bandgap (eV)")
plt.ylabel("Predicted Bandgap (eV)")
plt.show()